<a href="https://colab.research.google.com/github/danielmoraesdelima/mineracao_texto_web/blob/main/Projeto/TrabalhoFinal_Mineracao_de_Texto_Web_DanielMoraes_EmDesenvolvimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### TRABALHO FINAL DA DISCIPLINA − MINERAÇÃO DE TEXTO E DA WEB – 2021.1

Aluno: Daniel Moraes

Objetivos: Desenvolver um classificador para a tarefa de detecção de inferência entre duas sentenças em linguagem natural. Dada duas sentenças em linguagem natural (Português), o classificador deve definir a relação de inferência entre ambas:

Neutra
Implicação (A -> B): posso inferior B a partir de A


Contradição: A contradiz B, ou B contradiz A

Será fornecido um dataset com 9000 pares de sentenças rotuladas (com os resultados), para treinamento e validação.

As equipes serão avaliadas pelos entregáveis e pelos resultados a partir de um conjunto de teste (dados não vistos – Held-Out Data) com 1000 pares de sentenças.

#### Produtos Entregáveis:

Pré-processamento, Steemer, Lematização e POS Tagger, NER, etc (31/05/2021)

Vetorização das Sentenças (features)

Model Fitted e Resultados do processo de treinamento (28/06/2021)

Execução com Held-Out-Data (Dados não vistos) (12/07/2021)

Artigo resumo com 4 paginas (12/07/2021)

Apresentação Oral e Entrega do artigo: 05/07/2021 e 12/07/2021

In [ ]:
# Biblioteca para Manipulação de Tabelas
import pandas as pd
from pandas import Series, DataFrame

# Biblioteca para criar Gráficos
import matplotlib.pyplot as plt

# Biblioteca Arrays e Matrizes Multidimensionais
import numpy as np

#Biblioteca para visualização de dados estatísticos
import seaborn as sns
sns.set()

# Biblioteca para o Modelo
import sklearn as sk

#Models
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.seasonal import seasonal_decompose

#Definindo a localizacao
import locale  
from babel.numbers import format_decimal
locale.setlocale(locale.LC_MONETARY, 'en_US.UTF-8')

#Pacotes NLTK
import nltk
import re
import string
nltk.download('maxent_ne_chunker', quiet=True)
nltk.download('words', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('rslp', quiet=True)
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords as sw
from contextlib import redirect_stdout
from nltk.stem import PorterStemmer
from nltk.stem import RSLPStemmer
from nltk.tokenize import RegexpTokenizer 
from nltk.tokenize import WordPunctTokenizer
from nltk.tokenize import TweetTokenizer 
from nltk.stem import WordNetLemmatizer 
from nltk.stem import RSLPStemmer
import os
from pprint import pprint
import joblib
#Download das stopwords
with redirect_stdout(open(os.devnull, "w")):
    nltk.download("stopwords", quiet=True) 
    nltk.download('punkt', quiet=True)
    
import spacy 
#!python -m spacy download en_core_web_sm
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
#Importando o Dataset
#df_dados = pd.read_csv('C:\\Users\\Avell\\.conda\\envs\\Unifor\\PRJ\\MT\\Projeto\\Arquivos\\Dataset_SICK_BR_Treinamento.csv', delimiter=';', encoding= 'utf-8')
df = pd.read_csv('C:\\Users\\Avell\\.conda\\envs\\Unifor\\PRJ\\MT\\Projeto\\Arquivos\\Dataset_SICK_BR_Treinamento.csv', delimiter=';')

#Deletando colunas em portugues pois sua acentuação está errada, trabalharei nas sentenças em ingles
df.drop(['sentence_A','sentence_B'], axis=1, inplace=True)

#conversão da coluna 'pair_ID' de inteiro para string
df['pair_ID'] = df['pair_ID'].astype('str')

#Visualização do Cabeçalho dos dados
df.head()

,pair_ID,entailment_label,relatedness_score,entailment_AB,entailment_BA,sentence_A_original,sentence_B_original,sentence_A_dataset,sentence_B_dataset
0,1,NEUTRAL,4.5,A_neutral_B,B_neutral_A,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,FLICKR,FLICKR
1,2,NEUTRAL,3.2,A_contradicts_B,B_neutral_A,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR
2,3,ENTAILMENT,4.7,A_entails_B,B_entails_A,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,FLICKR,FLICKR
3,4,CONTRADICTION,3.6,A_contradicts_B,B_contradicts_A,The young boys are playing outdoors and the ma...,There is no boy playing outdoors and there is ...,FLICKR,FLICKR
4,5,NEUTRAL,3.4,A_neutral_B,B_neutral_A,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR


In [ ]:
#Visualização das informações do Dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8840 entries, 0 to 8839
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   pair_ID              8840 non-null   object 
 1   entailment_label     8840 non-null   object 
 2   relatedness_score    8840 non-null   float64
 3   entailment_AB        8840 non-null   object 
 4   entailment_BA        8840 non-null   object 
 5   sentence_A_original  8840 non-null   object 
 6   sentence_B_original  8840 non-null   object 
 7   sentence_A_dataset   8840 non-null   object 
 8   sentence_B_dataset   8840 non-null   object 
dtypes: float64(1), object(8)
memory usage: 621.7+ KB


#### Pré-processamento, Steemer, Lematização e POS Tagger, NER, etc (31/05/2021)

#### Tratamento inicial do texto

Converte o texto de cada sentença, separadamente, em minúsculo e remove espaços e tabulações extras. O resultado é guardado no DataFrame referente a cada sentenca em uma nova coluna.

In [ ]:
#Função para tratar o texto
def tratamento_texto(sentence):
  
  #remove as quebras de linha
  sentence = re.sub(r'\n', '', sentence)
  #substitui tabulações por um espaço em branco
  sentence = re.sub(r'\t', ' ', sentence)
  #substitui um ou mais espaços em branco por um espaço
  sentence= re.sub(r'\s+', ' ', sentence, flags=re.I)
  #&amp;
  #remove aspas e apóstofres
  sentence = re.sub('["‘’“”…]', '', sentence)
  return sentence

#cria uma nova coluna no dataframe 'sentence' com cada sentence tokenizado
df['texto_B_tratado'] = df['sentence_B_original'].apply(tratamento_texto)
df['texto_A_tratado'] = df['sentence_A_original'].apply(tratamento_texto)
#Exibe o resultado
df[df.columns[::-1]].head()

,texto_A_tratado,texto_B_tratado,sentence_B_dataset,sentence_A_dataset,sentence_B_original,sentence_A_original,entailment_BA,entailment_AB,relatedness_score,entailment_label,pair_ID
0,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,FLICKR,FLICKR,A group of boys in a yard is playing and a man...,A group of kids is playing in a yard and an ol...,B_neutral_A,A_neutral_B,4.5,NEUTRAL,1
1,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,A group of children is playing in the house an...,B_neutral_A,A_contradicts_B,3.2,NEUTRAL,2
2,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,FLICKR,FLICKR,The kids are playing outdoors near a man with ...,The young boys are playing outdoors and the ma...,B_entails_A,A_entails_B,4.7,ENTAILMENT,3
3,The young boys are playing outdoors and the ma...,There is no boy playing outdoors and there is ...,FLICKR,FLICKR,There is no boy playing outdoors and there is ...,The young boys are playing outdoors and the ma...,B_contradicts_A,A_contradicts_B,3.6,CONTRADICTION,4
4,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,The kids are playing outdoors near a man with ...,B_neutral_A,A_neutral_B,3.4,NEUTRAL,5


#### Separação em sentenças

Separa cada texto em sentenças e os coloca no DataFrame, atualizando a coluna

In [ ]:
#Funcao para separar as sentenças
def separa_sentencas(texto):
  lista_sentencas = sent_tokenize(texto)
  nova_lista = []
  for sent in lista_sentencas:
    nova_lista.append(sent.strip())
  return nova_lista

#cria uma nova coluna no dataframe 'sentence' com cada sentence tokenizado
df['texto_B_em_sentencas'] = df['texto_B_tratado'].apply(separa_sentencas)
df['texto_A_em_sentencas'] = df['texto_A_tratado'].apply(separa_sentencas)
#Exibe o resultado
df[df.columns[::-1]].head()

,texto_A_em_sentencas,texto_B_em_sentencas,texto_A_tratado,texto_B_tratado,sentence_B_dataset,sentence_A_dataset,sentence_B_original,sentence_A_original,entailment_BA,entailment_AB,relatedness_score,entailment_label,pair_ID
0,[A group of kids is playing in a yard and an o...,[A group of boys in a yard is playing and a ma...,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,FLICKR,FLICKR,A group of boys in a yard is playing and a man...,A group of kids is playing in a yard and an ol...,B_neutral_A,A_neutral_B,4.5,NEUTRAL,1
1,[A group of children is playing in the house a...,[A group of kids is playing in a yard and an o...,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,A group of children is playing in the house an...,B_neutral_A,A_contradicts_B,3.2,NEUTRAL,2
2,[The young boys are playing outdoors and the m...,[The kids are playing outdoors near a man with...,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,FLICKR,FLICKR,The kids are playing outdoors near a man with ...,The young boys are playing outdoors and the ma...,B_entails_A,A_entails_B,4.7,ENTAILMENT,3
3,[The young boys are playing outdoors and the m...,[There is no boy playing outdoors and there is...,The young boys are playing outdoors and the ma...,There is no boy playing outdoors and there is ...,FLICKR,FLICKR,There is no boy playing outdoors and there is ...,The young boys are playing outdoors and the ma...,B_contradicts_A,A_contradicts_B,3.6,CONTRADICTION,4
4,[The kids are playing outdoors near a man with...,[A group of kids is playing in a yard and an o...,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,The kids are playing outdoors near a man with ...,B_neutral_A,A_neutral_B,3.4,NEUTRAL,5


#### Tokenização

Reúne as sentenças em uma string única e realiza a tokenização do texto. O resultado pode ser observado em uma nova coluna.

In [ ]:
#Funcao para tokenizar - separar as sentenças
def tokeniza_sentenca(lista_sentencas):
  sentencas_unidas = " ".join(w for w in lista_sentencas)
  sentence = sent_tokenize(sentencas_unidas)
  tokens = word_tokenize(sentence[0])
  return tokens

#cria uma nova coluna no dataframe 'sentence' com cada sentence tokenizado
df['texto_B_tokenizado'] = df['texto_B_em_sentencas'].apply(tokeniza_sentenca)
df['texto_A_tokenizado'] = df['texto_A_em_sentencas'].apply(tokeniza_sentenca)
#Exibe o resultado
df[df.columns[::-1]].head()

,texto_A_tokenizado,texto_B_tokenizado,texto_A_em_sentencas,texto_B_em_sentencas,texto_A_tratado,texto_B_tratado,sentence_B_dataset,sentence_A_dataset,sentence_B_original,sentence_A_original,entailment_BA,entailment_AB,relatedness_score,entailment_label,pair_ID
0,"[A, group, of, kids, is, playing, in, a, yard,...","[A, group, of, boys, in, a, yard, is, playing,...",[A group of kids is playing in a yard and an o...,[A group of boys in a yard is playing and a ma...,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,FLICKR,FLICKR,A group of boys in a yard is playing and a man...,A group of kids is playing in a yard and an ol...,B_neutral_A,A_neutral_B,4.5,NEUTRAL,1
1,"[A, group, of, children, is, playing, in, the,...","[A, group, of, kids, is, playing, in, a, yard,...",[A group of children is playing in the house a...,[A group of kids is playing in a yard and an o...,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,A group of children is playing in the house an...,B_neutral_A,A_contradicts_B,3.2,NEUTRAL,2
2,"[The, young, boys, are, playing, outdoors, and...","[The, kids, are, playing, outdoors, near, a, m...",[The young boys are playing outdoors and the m...,[The kids are playing outdoors near a man with...,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,FLICKR,FLICKR,The kids are playing outdoors near a man with ...,The young boys are playing outdoors and the ma...,B_entails_A,A_entails_B,4.7,ENTAILMENT,3
3,"[The, young, boys, are, playing, outdoors, and...","[There, is, no, boy, playing, outdoors, and, t...",[The young boys are playing outdoors and the m...,[There is no boy playing outdoors and there is...,The young boys are playing outdoors and the ma...,There is no boy playing outdoors and there is ...,FLICKR,FLICKR,There is no boy playing outdoors and there is ...,The young boys are playing outdoors and the ma...,B_contradicts_A,A_contradicts_B,3.6,CONTRADICTION,4
4,"[The, kids, are, playing, outdoors, near, a, m...","[A, group, of, kids, is, playing, in, a, yard,...",[The kids are playing outdoors near a man with...,[A group of kids is playing in a yard and an o...,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,The kids are playing outdoors near a man with ...,B_neutral_A,A_neutral_B,3.4,NEUTRAL,5


#### Remoção de stop words

Remove da lista de tokens de cada texto as stop words da língua inglesa e pontuações.

In [ ]:
#Remover palavras que não são importantes para a contabilizaçao
def remove_stop_words(lista_tokens):
  stopwords = sw.words('english')
  stop_words = set(stopwords + list(string.punctuation))
  tokens = [w for w in lista_tokens if not w in stop_words]
  return tokens

#cria uma nova coluna no dataframe sem stopwords
df['tokens_sem_stopwords_A'] = df['texto_B_tokenizado'].apply(remove_stop_words)
df['tokens_sem_stopwords_B'] = df['texto_A_tokenizado'].apply(remove_stop_words)
#Exibe o resultado
df[df.columns[::-1]].head()

,tokens_sem_stopwords_B,tokens_sem_stopwords_A,texto_A_tokenizado,texto_B_tokenizado,texto_A_em_sentencas,texto_B_em_sentencas,texto_A_tratado,texto_B_tratado,sentence_B_dataset,sentence_A_dataset,sentence_B_original,sentence_A_original,entailment_BA,entailment_AB,relatedness_score,entailment_label,pair_ID
0,"[A, group, kids, playing, yard, old, man, stan...","[A, group, boys, yard, playing, man, standing,...","[A, group, of, kids, is, playing, in, a, yard,...","[A, group, of, boys, in, a, yard, is, playing,...",[A group of kids is playing in a yard and an o...,[A group of boys in a yard is playing and a ma...,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,FLICKR,FLICKR,A group of boys in a yard is playing and a man...,A group of kids is playing in a yard and an ol...,B_neutral_A,A_neutral_B,4.5,NEUTRAL,1
1,"[A, group, children, playing, house, man, stan...","[A, group, kids, playing, yard, old, man, stan...","[A, group, of, children, is, playing, in, the,...","[A, group, of, kids, is, playing, in, a, yard,...",[A group of children is playing in the house a...,[A group of kids is playing in a yard and an o...,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,A group of children is playing in the house an...,B_neutral_A,A_contradicts_B,3.2,NEUTRAL,2
2,"[The, young, boys, playing, outdoors, man, smi...","[The, kids, playing, outdoors, near, man, smile]","[The, young, boys, are, playing, outdoors, and...","[The, kids, are, playing, outdoors, near, a, m...",[The young boys are playing outdoors and the m...,[The kids are playing outdoors near a man with...,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,FLICKR,FLICKR,The kids are playing outdoors near a man with ...,The young boys are playing outdoors and the ma...,B_entails_A,A_entails_B,4.7,ENTAILMENT,3
3,"[The, young, boys, playing, outdoors, man, smi...","[There, boy, playing, outdoors, man, smiling]","[The, young, boys, are, playing, outdoors, and...","[There, is, no, boy, playing, outdoors, and, t...",[The young boys are playing outdoors and the m...,[There is no boy playing outdoors and there is...,The young boys are playing outdoors and the ma...,There is no boy playing outdoors and there is ...,FLICKR,FLICKR,There is no boy playing outdoors and there is ...,The young boys are playing outdoors and the ma...,B_contradicts_A,A_contradicts_B,3.6,CONTRADICTION,4
4,"[The, kids, playing, outdoors, near, man, smile]","[A, group, kids, playing, yard, old, man, stan...","[The, kids, are, playing, outdoors, near, a, m...","[A, group, of, kids, is, playing, in, a, yard,...",[The kids are playing outdoors near a man with...,[A group of kids is playing in a yard and an o...,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,The kids are playing outdoors near a man with ...,B_neutral_A,A_neutral_B,3.4,NEUTRAL,5


#### Part-of-speech tagging

processo de marcar uma palavra em um texto como correspondendo a uma parte específica do discurso, com base em sua definição e em seu contexto

In [ ]:
#Funcao para o POS TAGGING 
def Postagging(sentence):
    phrase = []
    for word in sentence:
        phrase.append(nltk.pos_tag(word.lower()))
    return phrase

#cria uma nova coluna no dataframe com o Postagging
df['postagging_A'] = df['tokens_sem_stopwords_B'].apply(Postagging)
df['postagging_B'] = df['tokens_sem_stopwords_A'].apply(Postagging)
#Exibe o resultado
df[df.columns[::-1]].head()

,postagging_B,postagging_A,tokens_sem_stopwords_B,tokens_sem_stopwords_A,texto_A_tokenizado,texto_B_tokenizado,texto_A_em_sentencas,texto_B_em_sentencas,texto_A_tratado,texto_B_tratado,sentence_B_dataset,sentence_A_dataset,sentence_B_original,sentence_A_original,entailment_BA,entailment_AB,relatedness_score,entailment_label,pair_ID
0,"[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[A, group, kids, playing, yard, old, man, stan...","[A, group, boys, yard, playing, man, standing,...","[A, group, of, kids, is, playing, in, a, yard,...","[A, group, of, boys, in, a, yard, is, playing,...",[A group of kids is playing in a yard and an o...,[A group of boys in a yard is playing and a ma...,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,FLICKR,FLICKR,A group of boys in a yard is playing and a man...,A group of kids is playing in a yard and an ol...,B_neutral_A,A_neutral_B,4.5,NEUTRAL,1
1,"[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[A, group, children, playing, house, man, stan...","[A, group, kids, playing, yard, old, man, stan...","[A, group, of, children, is, playing, in, the,...","[A, group, of, kids, is, playing, in, a, yard,...",[A group of children is playing in the house a...,[A group of kids is playing in a yard and an o...,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,A group of children is playing in the house an...,B_neutral_A,A_contradicts_B,3.2,NEUTRAL,2
2,"[[(t, NN), (h, NN), (e, NN)], [(k, NN), (i, NN...","[[(t, NN), (h, NN), (e, NN)], [(y, NN), (o, MD...","[The, young, boys, playing, outdoors, man, smi...","[The, kids, playing, outdoors, near, man, smile]","[The, young, boys, are, playing, outdoors, and...","[The, kids, are, playing, outdoors, near, a, m...",[The young boys are playing outdoors and the m...,[The kids are playing outdoors near a man with...,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,FLICKR,FLICKR,The kids are playing outdoors near a man with ...,The young boys are playing outdoors and the ma...,B_entails_A,A_entails_B,4.7,ENTAILMENT,3
3,"[[(t, NN), (h, NN), (e, NN), (r, NN), (e, NN)]...","[[(t, NN), (h, NN), (e, NN)], [(y, NN), (o, MD...","[The, young, boys, playing, outdoors, man, smi...","[There, boy, playing, outdoors, man, smiling]","[The, young, boys, are, playing, outdoors, and...","[There, is, no, boy, playing, outdoors, and, t...",[The young boys are playing outdoors and the m...,[There is no boy playing outdoors and there is...,The young boys are playing outdoors and the ma...,There is no boy playing outdoors and there is ...,FLICKR,FLICKR,There is no boy playing outdoors and there is ...,The young boys are playing outdoors and the ma...,B_contradicts_A,A_contradicts_B,3.6,CONTRADICTION,4
4,"[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[[(t, NN), (h, NN), (e, NN)], [(k, NN), (i, NN...","[The, kids, playing, outdoors, near, man, smile]","[A, group, kids, playing, yard, old, man, stan...","[The, kids, are, playing, outdoors, near, a, m...","[A, group, of, kids, is, playing, in, a, yard,...",[The kids are playing outdoors near a man with...,[A group of kids is playing in a yard and an o...,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,The kids are playing outdoors near a man with ...,B_neutral_A,A_neutral_B,3.4,NEUTRAL,5


#### Stemming

Processo de extração do radical ou raiz de um TOKEN

In [ ]:
#Funcao para stemmingzar (tratar o radical da palavra, até a origem)
def Stemming(sentence):
    stemmer = RSLPStemmer()
    phrase = []
    for word in sentence:
        phrase.append(stemmer.stem(word.lower()))
    return phrase

#cria uma nova coluna no dataframe com a raiz de tokens
df['raiz_tokens_A'] = df['tokens_sem_stopwords_B'].apply(Stemming)
df['raiz_tokens_B'] = df['tokens_sem_stopwords_A'].apply(Stemming)
#Exibe o resultado
df[df.columns[::-1]].head()

,raiz_tokens_B,raiz_tokens_A,postagging_B,postagging_A,tokens_sem_stopwords_B,tokens_sem_stopwords_A,texto_A_tokenizado,texto_B_tokenizado,texto_A_em_sentencas,texto_B_em_sentencas,...,texto_B_tratado,sentence_B_dataset,sentence_A_dataset,sentence_B_original,sentence_A_original,entailment_BA,entailment_AB,relatedness_score,entailment_label,pair_ID
0,"[a, group, boy, yard, playing, man, standing, ...","[a, group, kid, playing, yard, old, man, stand...","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[A, group, kids, playing, yard, old, man, stan...","[A, group, boys, yard, playing, man, standing,...","[A, group, of, kids, is, playing, in, a, yard,...","[A, group, of, boys, in, a, yard, is, playing,...",[A group of kids is playing in a yard and an o...,[A group of boys in a yard is playing and a ma...,...,A group of boys in a yard is playing and a man...,FLICKR,FLICKR,A group of boys in a yard is playing and a man...,A group of kids is playing in a yard and an ol...,B_neutral_A,A_neutral_B,4.5,NEUTRAL,1
1,"[a, group, kid, playing, yard, old, man, stand...","[a, group, children, playing, hous, man, stand...","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[A, group, children, playing, house, man, stan...","[A, group, kids, playing, yard, old, man, stan...","[A, group, of, children, is, playing, in, the,...","[A, group, of, kids, is, playing, in, a, yard,...",[A group of children is playing in the house a...,[A group of kids is playing in a yard and an o...,...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,A group of children is playing in the house an...,B_neutral_A,A_contradicts_B,3.2,NEUTRAL,2
2,"[the, kid, playing, outdo, ne, man, smil]","[the, young, boy, playing, outdo, man, smiling...","[[(t, NN), (h, NN), (e, NN)], [(k, NN), (i, NN...","[[(t, NN), (h, NN), (e, NN)], [(y, NN), (o, MD...","[The, young, boys, playing, outdoors, man, smi...","[The, kids, playing, outdoors, near, man, smile]","[The, young, boys, are, playing, outdoors, and...","[The, kids, are, playing, outdoors, near, a, m...",[The young boys are playing outdoors and the m...,[The kids are playing outdoors near a man with...,...,The kids are playing outdoors near a man with ...,FLICKR,FLICKR,The kids are playing outdoors near a man with ...,The young boys are playing outdoors and the ma...,B_entails_A,A_entails_B,4.7,ENTAILMENT,3
3,"[ther, boy, playing, outdo, man, smiling]","[the, young, boy, playing, outdo, man, smiling...","[[(t, NN), (h, NN), (e, NN), (r, NN), (e, NN)]...","[[(t, NN), (h, NN), (e, NN)], [(y, NN), (o, MD...","[The, young, boys, playing, outdoors, man, smi...","[There, boy, playing, outdoors, man, smiling]","[The, young, boys, are, playing, outdoors, and...","[There, is, no, boy, playing, outdoors, and, t...",[The young boys are playing outdoors and the m...,[There is no boy playing outdoors and there is...,...,There is no boy playing outdoors and there is ...,FLICKR,FLICKR,There is no boy playing outdoors and there is ...,The young boys are playing outdoors and the ma...,B_contradicts_A,A_contradicts_B,3.6,CONTRADICTION,4
4,"[a, group, kid, playing, yard, old, man, stand...","[the, kid, playing, outdo, ne, man, smil]","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[[(t, NN), (h, NN), (e, NN)], [(k, NN), (i, NN...","[The, kids, playing, outdoors, near, man, smile]","[A, group, kids, playing, yard, old, man, stan...","[The, kids, are, playing, outdoors, near, a, m...","[A, group, of, kids, is, playing, in, a, yard,...",[The kids are playing outdoors near a man with...,[A group of kids is playing in a yard and an o...,...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,The kids are playing outdoors near a man with ...,B_neutral_A,A_neutral_B,3.4,NEUTRAL,5


#### Lematizar

Processo de agrupar formas flexionadas em uma só palavra com similar.

In [ ]:
#Funcao para Lematizar: processo de agrupar formas flexionadas em uma só palavra com similar.
def Lematizar(sentence):
    lemmatizer = WordNetLemmatizer()
    phrase = []
    for word in sentence:
        phrase.append(lemmatizer.lemmatize(word.lower()))
    return phrase

#cria uma nova coluna com o texto com formas flexionadas
df['texto_lematizado_A'] = df['raiz_tokens_B'].apply(Lematizar)
df['texto_lematizado_B'] = df['raiz_tokens_A'].apply(Lematizar)
#Exibe o resultado
df[df.columns[::-1]].head()

,texto_lematizado_B,texto_lematizado_A,raiz_tokens_B,raiz_tokens_A,postagging_B,postagging_A,tokens_sem_stopwords_B,tokens_sem_stopwords_A,texto_A_tokenizado,texto_B_tokenizado,...,texto_B_tratado,sentence_B_dataset,sentence_A_dataset,sentence_B_original,sentence_A_original,entailment_BA,entailment_AB,relatedness_score,entailment_label,pair_ID
0,"[a, group, kid, playing, yard, old, man, stand...","[a, group, boy, yard, playing, man, standing, ...","[a, group, boy, yard, playing, man, standing, ...","[a, group, kid, playing, yard, old, man, stand...","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[A, group, kids, playing, yard, old, man, stan...","[A, group, boys, yard, playing, man, standing,...","[A, group, of, kids, is, playing, in, a, yard,...","[A, group, of, boys, in, a, yard, is, playing,...",...,A group of boys in a yard is playing and a man...,FLICKR,FLICKR,A group of boys in a yard is playing and a man...,A group of kids is playing in a yard and an ol...,B_neutral_A,A_neutral_B,4.5,NEUTRAL,1
1,"[a, group, child, playing, hous, man, standing...","[a, group, kid, playing, yard, old, man, stand...","[a, group, kid, playing, yard, old, man, stand...","[a, group, children, playing, hous, man, stand...","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[A, group, children, playing, house, man, stan...","[A, group, kids, playing, yard, old, man, stan...","[A, group, of, children, is, playing, in, the,...","[A, group, of, kids, is, playing, in, a, yard,...",...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,A group of children is playing in the house an...,B_neutral_A,A_contradicts_B,3.2,NEUTRAL,2
2,"[the, young, boy, playing, outdo, man, smiling...","[the, kid, playing, outdo, ne, man, smil]","[the, kid, playing, outdo, ne, man, smil]","[the, young, boy, playing, outdo, man, smiling...","[[(t, NN), (h, NN), (e, NN)], [(k, NN), (i, NN...","[[(t, NN), (h, NN), (e, NN)], [(y, NN), (o, MD...","[The, young, boys, playing, outdoors, man, smi...","[The, kids, playing, outdoors, near, man, smile]","[The, young, boys, are, playing, outdoors, and...","[The, kids, are, playing, outdoors, near, a, m...",...,The kids are playing outdoors near a man with ...,FLICKR,FLICKR,The kids are playing outdoors near a man with ...,The young boys are playing outdoors and the ma...,B_entails_A,A_entails_B,4.7,ENTAILMENT,3
3,"[the, young, boy, playing, outdo, man, smiling...","[ther, boy, playing, outdo, man, smiling]","[ther, boy, playing, outdo, man, smiling]","[the, young, boy, playing, outdo, man, smiling...","[[(t, NN), (h, NN), (e, NN), (r, NN), (e, NN)]...","[[(t, NN), (h, NN), (e, NN)], [(y, NN), (o, MD...","[The, young, boys, playing, outdoors, man, smi...","[There, boy, playing, outdoors, man, smiling]","[The, young, boys, are, playing, outdoors, and...","[There, is, no, boy, playing, outdoors, and, t...",...,There is no boy playing outdoors and there is ...,FLICKR,FLICKR,There is no boy playing outdoors and there is ...,The young boys are playing outdoors and the ma...,B_contradicts_A,A_contradicts_B,3.6,CONTRADICTION,4
4,"[the, kid, playing, outdo, ne, man, smil]","[a, group, kid, playing, yard, old, man, stand...","[a, group, kid, playing, yard, old, man, stand...","[the, kid, playing, outdo, ne, man, smil]","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[[(t, NN), (h, NN), (e, NN)], [(k, NN), (i, NN...","[The, kids, playing, outdoors, near, man, smile]","[A, group, kids, playing, yard, old, man, stan...","[The, kids, are, playing, outdoors, near, a, m...","[A, group, of, kids, is, playing, in, a, yard,...",...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,The kids are playing outdoors near a man with ...,B_neutral_A,A_neutral_B,3.4,NEUTRAL,5


#### Reconhecimento de Entidades Mencionadas (REM) ou Named-Entity Recognition (NER)

NER usando pacote SpaCy

In [ ]:
#Funcao para aplicar o LER nas setencas
def Ner(sentence):
    doc = nlp(sentence) 
    phrase = []
    for ent in doc.ents:
        phrase.append(ent.text, ent.start_char, ent.end_char, ent.label_)
    return phrase

#cria uma nova coluna com o texto com as entidades identificadas
df['NER_A'] = df['texto_lematizado_B'].apply(Lematizar)
df['NER_B'] = df['texto_lematizado_A'].apply(Lematizar)
#Exibe o resultado
df[df.columns[::-1]].head()

,NER_B,NER_A,texto_lematizado_B,texto_lematizado_A,raiz_tokens_B,raiz_tokens_A,postagging_B,postagging_A,tokens_sem_stopwords_B,tokens_sem_stopwords_A,...,texto_B_tratado,sentence_B_dataset,sentence_A_dataset,sentence_B_original,sentence_A_original,entailment_BA,entailment_AB,relatedness_score,entailment_label,pair_ID
0,"[a, group, boy, yard, playing, man, standing, ...","[a, group, kid, playing, yard, old, man, stand...","[a, group, kid, playing, yard, old, man, stand...","[a, group, boy, yard, playing, man, standing, ...","[a, group, boy, yard, playing, man, standing, ...","[a, group, kid, playing, yard, old, man, stand...","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[A, group, kids, playing, yard, old, man, stan...","[A, group, boys, yard, playing, man, standing,...",...,A group of boys in a yard is playing and a man...,FLICKR,FLICKR,A group of boys in a yard is playing and a man...,A group of kids is playing in a yard and an ol...,B_neutral_A,A_neutral_B,4.5,NEUTRAL,1
1,"[a, group, kid, playing, yard, old, man, stand...","[a, group, child, playing, hous, man, standing...","[a, group, child, playing, hous, man, standing...","[a, group, kid, playing, yard, old, man, stand...","[a, group, kid, playing, yard, old, man, stand...","[a, group, children, playing, hous, man, stand...","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[A, group, children, playing, house, man, stan...","[A, group, kids, playing, yard, old, man, stan...",...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,A group of children is playing in the house an...,B_neutral_A,A_contradicts_B,3.2,NEUTRAL,2
2,"[the, kid, playing, outdo, ne, man, smil]","[the, young, boy, playing, outdo, man, smiling...","[the, young, boy, playing, outdo, man, smiling...","[the, kid, playing, outdo, ne, man, smil]","[the, kid, playing, outdo, ne, man, smil]","[the, young, boy, playing, outdo, man, smiling...","[[(t, NN), (h, NN), (e, NN)], [(k, NN), (i, NN...","[[(t, NN), (h, NN), (e, NN)], [(y, NN), (o, MD...","[The, young, boys, playing, outdoors, man, smi...","[The, kids, playing, outdoors, near, man, smile]",...,The kids are playing outdoors near a man with ...,FLICKR,FLICKR,The kids are playing outdoors near a man with ...,The young boys are playing outdoors and the ma...,B_entails_A,A_entails_B,4.7,ENTAILMENT,3
3,"[ther, boy, playing, outdo, man, smiling]","[the, young, boy, playing, outdo, man, smiling...","[the, young, boy, playing, outdo, man, smiling...","[ther, boy, playing, outdo, man, smiling]","[ther, boy, playing, outdo, man, smiling]","[the, young, boy, playing, outdo, man, smiling...","[[(t, NN), (h, NN), (e, NN), (r, NN), (e, NN)]...","[[(t, NN), (h, NN), (e, NN)], [(y, NN), (o, MD...","[The, young, boys, playing, outdoors, man, smi...","[There, boy, playing, outdoors, man, smiling]",...,There is no boy playing outdoors and there is ...,FLICKR,FLICKR,There is no boy playing outdoors and there is ...,The young boys are playing outdoors and the ma...,B_contradicts_A,A_contradicts_B,3.6,CONTRADICTION,4
4,"[a, group, kid, playing, yard, old, man, stand...","[the, kid, playing, outdo, ne, man, smil]","[the, kid, playing, outdo, ne, man, smil]","[a, group, kid, playing, yard, old, man, stand...","[a, group, kid, playing, yard, old, man, stand...","[the, kid, playing, outdo, ne, man, smil]","[[(a, DT)], [(g, NN), (r, NN), (o, NN), (u, JJ...","[[(t, NN), (h, NN), (e, NN)], [(k, NN), (i, NN...","[The, kids, playing, outdoors, near, man, smile]","[A, group, kids, playing, yard, old, man, stan...",...,A group of kids is playing in a yard and an ol...,FLICKR,FLICKR,A group of kids is playing in a yard and an ol...,The kids are playing outdoors near a man with ...,B_neutral_A,A_neutral_B,3.4,NEUTRAL,5
